In [95]:
import pandas as pd
from string import punctuation
!pip install stop_words
from stop_words import get_stop_words
!pip install pymorphy2
from pymorphy2 import MorphAnalyzer
import re
import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, SimpleRNN, LSTM, GRU, Masking, Flatten
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
# from keras.objectives import categorical_crossentropy
from keras.callbacks import EarlyStopping  



In [96]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [97]:
df_train = pd.read_csv("/content/gdrive/MyDrive/train.csv")
df_test = pd.read_csv("/content/gdrive/MyDrive/test.csv")
df_val = pd.read_csv("/content/gdrive/MyDrive/val.csv")

In [98]:
df_train

,id,text,class
0,0,@alisachachka не уезжаааааааай. :(❤ я тоже не ...,0
1,1,RT @GalyginVadim: Ребята и девчата!\nВсе в кин...,1
2,2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,0
3,3,RT @epupybobv: Хочется котлету по-киевски. Зап...,1
4,4,@KarineKurganova @Yess__Boss босапопа есбоса н...,1
...,...,...,...
181462,181462,Классным будет новый год..( http://t.co/LejaGu...,0
181463,181463,Видишь человека хочешь сказать ему привет . А ...,0
181464,181464,@Julia6_9Styles там длинный диагноз. Вкратце: ...,0
181465,181465,"@technoslav ух ты, а кому тот серенький кнопоч...",1


In [99]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df_train['text'] = df_train['text'].apply(preprocess_text)
df_val['text'] = df_val['text'].apply(preprocess_text)
df_test['text'] = df_test['text'].apply(preprocess_text)

In [100]:
text_corpus_train = df_train['text'].values
text_corpus_valid = df_val['text'].values
text_corpus_test = df_test['text'].values

In [101]:
tokenizer = Tokenizer(num_words=None, 
                     filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n',
                     lower = False, split = ' ')
tokenizer.fit_on_texts(text_corpus_train)

sequences_train = tokenizer.texts_to_sequences(text_corpus_train)
sequences_val = tokenizer.texts_to_sequences(text_corpus_valid)
sequences_test = tokenizer.texts_to_sequences(text_corpus_test)

word_count = len(tokenizer.index_word) + 1
training_length = max([len(i.split()) for i in text_corpus_train])

X_train = pad_sequences(sequences_train, maxlen=training_length)
X_valid = pad_sequences(sequences_val, maxlen=training_length)



In [102]:
y_train = df_train['class'].values
y_val = df_val['class'].values

In [103]:
def make_note(name, model, df):
    score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
    lenght = len(df.Name)
    df.loc[lenght] = name, score[1]
    return df

In [104]:
df_score = pd.DataFrame(columns=['Name', 'Score'])

In [105]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(SimpleRNN(64))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [106]:
model.summary()

Model: "sequential_32"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_32 (Embedding)     (None, 27, 30)            7743240   
_________________________________________________________________
masking_32 (Masking)         (None, 27, 30)            0         
_________________________________________________________________
simple_rnn_8 (SimpleRNN)     (None, 64)                6080      
_________________________________________________________________
dense_55 (Dense)             (None, 64)                4160      
_________________________________________________________________
dropout_31 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_56 (Dense)             (None, 1)                 65        
Total params: 7,753,545
Trainable params: 7,753,545
Non-trainable params: 0
___________________________________________

In [107]:
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
319/319 [==============================] - 24s 72ms/step - loss: 0.5567 - accuracy: 0.7022 - val_loss: 0.4926 - val_accuracy: 0.7579


In [108]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 0s 7ms/step - loss: 0.5082 - accuracy: 0.7432


Test score: 0.5081614255905151
Test accuracy: 0.7431997656822205


In [109]:
make_note('SimpleRNN', model, df_score)

45/45 [==============================] - 0s 6ms/step - loss: 0.5082 - accuracy: 0.7432


,Name,Score
0,SimpleRNN,0.7432


In [110]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(LSTM(64, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(X_train, y_train,
                    batch_size=2048,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
80/80 [==============================] - 21s 230ms/step - loss: 0.6198 - accuracy: 0.6524 - val_loss: 0.5168 - val_accuracy: 0.7443


In [111]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 1s 15ms/step - loss: 0.5280 - accuracy: 0.7335


Test score: 0.5279825925827026
Test accuracy: 0.7334567904472351


In [112]:
make_note('LSTM', model, df_score)

45/45 [==============================] - 1s 15ms/step - loss: 0.5280 - accuracy: 0.7335


,Name,Score
0,SimpleRNN,0.743200
1,LSTM,0.733457


In [113]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(GRU(64, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
319/319 [==============================] - 59s 175ms/step - loss: 0.5529 - accuracy: 0.7094 - val_loss: 0.5006 - val_accuracy: 0.7472


In [114]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 1s 14ms/step - loss: 0.5036 - accuracy: 0.7460


Test score: 0.5036002993583679
Test accuracy: 0.7460212707519531


In [115]:
make_note('GRU', model, df_score)

45/45 [==============================] - 1s 14ms/step - loss: 0.5036 - accuracy: 0.7460


,Name,Score
0,SimpleRNN,0.743200
1,LSTM,0.733457
2,GRU,0.746021


In [116]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(Conv1D(32, kernel_size=5, activation='relu'))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss')  

history = model.fit(X_train, y_train,
                    batch_size=2048,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
80/80 [==============================] - 3s 20ms/step - loss: 0.6261 - accuracy: 0.6340 - val_loss: 0.5242 - val_accuracy: 0.7351


In [117]:
model.summary()

Model: "sequential_35"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_35 (Embedding)     (None, 27, 30)            7743240   
_________________________________________________________________
masking_35 (Masking)         (None, 27, 30)            0         
_________________________________________________________________
conv1d_27 (Conv1D)           (None, 23, 32)            4832      
_________________________________________________________________
flatten_17 (Flatten)         (None, 736)               0         
_________________________________________________________________
dense_61 (Dense)             (None, 64)                47168     
_________________________________________________________________
dropout_34 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_62 (Dense)             (None, 1)               

In [118]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])
print(type(score[1]))

45/45 [==============================] - 0s 3ms/step - loss: 0.5454 - accuracy: 0.7189


Test score: 0.5453770160675049
Test accuracy: 0.718908429145813
<class 'float'>


In [119]:
make_note('1_Conv', model, df_score)

45/45 [==============================] - 0s 3ms/step - loss: 0.5454 - accuracy: 0.7189


,Name,Score
0,SimpleRNN,0.743200
1,LSTM,0.733457
2,GRU,0.746021
3,1_Conv,0.718908


In [129]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(Conv1D(64, kernel_size=7, activation='relu'))
model.add(Conv1D(128, kernel_size=5, activation='relu'))
model.add(Flatten())
model.add(Dense(1344, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='tanh'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss')  

history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=2,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/2
319/319 [==============================] - 8s 23ms/step - loss: 0.5477 - accuracy: 0.7082 - val_loss: 0.4849 - val_accuracy: 0.7585


In [130]:
model.summary()

Model: "sequential_39"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_39 (Embedding)     (None, 27, 30)            7743240   
_________________________________________________________________
masking_39 (Masking)         (None, 27, 30)            0         
_________________________________________________________________
conv1d_31 (Conv1D)           (None, 21, 64)            13504     
_________________________________________________________________
conv1d_32 (Conv1D)           (None, 17, 128)           41088     
_________________________________________________________________
flatten_19 (Flatten)         (None, 2176)              0         
_________________________________________________________________
dense_68 (Dense)             (None, 1344)              2925888   
_________________________________________________________________
dropout_38 (Dropout)         (None, 1344)            

In [122]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 0s 4ms/step - loss: 0.5137 - accuracy: 0.7358


Test score: 0.5137268304824829
Test accuracy: 0.7357933521270752


In [123]:
make_note('2_Conv', model, df_score)

45/45 [==============================] - 0s 4ms/step - loss: 0.5137 - accuracy: 0.7358


,Name,Score
0,SimpleRNN,0.743200
1,LSTM,0.733457
2,GRU,0.746021
3,1_Conv,0.718908
4,2_Conv,0.735793


In [124]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(Conv1D(32, kernel_size=7, activation='relu'))
model.add(LSTM(64, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss')  

history = model.fit(X_train, y_train,
                    batch_size=2048,
                    epochs=2,
                    verbose=1,
                    validation_split=0.1,
                    )

Epoch 1/2
80/80 [==============================] - 15s 151ms/step - loss: 0.6113 - accuracy: 0.6548 - val_loss: 0.5151 - val_accuracy: 0.7431
Epoch 2/2
80/80 [==============================] - 12s 145ms/step - loss: 0.3768 - accuracy: 0.8396 - val_loss: 0.5182 - val_accuracy: 0.7501


In [125]:
make_note('Conv_LSTM', model, df_score)

45/45 [==============================] - 1s 12ms/step - loss: 0.5548 - accuracy: 0.7354


,Name,Score
0,SimpleRNN,0.743200
1,LSTM,0.733457
2,GRU,0.746021
3,1_Conv,0.718908
4,2_Conv,0.735793
5,Conv_LSTM,0.735352


In [134]:
df_score.sort_values(by=['Score'])

,Name,Score
3,1_Conv,0.718908
1,LSTM,0.733457
5,Conv_LSTM,0.735352
4,2_Conv,0.735793
0,SimpleRNN,0.743200
6,2_Conv_LSTM,0.744478
2,GRU,0.746021


In [127]:
model.summary()

Model: "sequential_37"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_37 (Embedding)     (None, 27, 30)            7743240   
_________________________________________________________________
masking_37 (Masking)         (None, 27, 30)            0         
_________________________________________________________________
conv1d_29 (Conv1D)           (None, 21, 32)            6752      
_________________________________________________________________
lstm_7 (LSTM)                (None, 64)                24832     
_________________________________________________________________
dense_66 (Dense)             (None, 64)                4160      
_________________________________________________________________
dropout_37 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_67 (Dense)             (None, 1)               

In [167]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(Conv1D(64, kernel_size=11, activation='relu'))
model.add(Conv1D(128, kernel_size=7, activation='relu'))
model.add(LSTM(256, recurrent_dropout=0.2))
model.add(Dense(512, activation='tanh'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='tanh'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss')  

history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=2,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/2
319/319 [==============================] - 33s 94ms/step - loss: 0.5451 - accuracy: 0.7084 - val_loss: 0.4856 - val_accuracy: 0.7579


In [168]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 1s 13ms/step - loss: 0.5078 - accuracy: 0.7439


Test score: 0.5078067779541016
Test accuracy: 0.7439492344856262


In [164]:
model.summary()

Model: "sequential_67"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_67 (Embedding)     (None, 27, 30)            7743240   
_________________________________________________________________
masking_67 (Masking)         (None, 27, 30)            0         
_________________________________________________________________
conv1d_87 (Conv1D)           (None, 17, 64)            21184     
_________________________________________________________________
conv1d_88 (Conv1D)           (None, 11, 128)           57472     
_________________________________________________________________
gru_4 (GRU)                  (None, 128)               99072     
_________________________________________________________________
dense_163 (Dense)            (None, 512)               66048     
_________________________________________________________________
dropout_92 (Dropout)         (None, 512)             

In [133]:
make_note('2_Conv_LSTM', model, df_score)

45/45 [==============================] - 1s 16ms/step - loss: 0.5096 - accuracy: 0.7445


,Name,Score
0,SimpleRNN,0.743200
1,LSTM,0.733457
2,GRU,0.746021
3,1_Conv,0.718908
4,2_Conv,0.735793
5,Conv_LSTM,0.735352
6,2_Conv_LSTM,0.744478


Слой SimpleRNN справляется так же, как и два слоя свертки + lstm. Выше таблица с результатами.